# Scrape APIs for options and risk-free rates data

Suggestions for looking for API sources:
- BoE 
- ONS 
- UK Gov
- World Bank
- IMF
- UN
- FiscalData.Treasury.gov

In [116]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import requests
from datetime import datetime as dt
import json
import re

In [117]:
## Options data

## API URL
url_marketData = "https://api.marketdata.app/v1/options/quotes/"

## Option symbol for AAPL Jan 2025 150.000 call
optSymbol = "AAPL250117C00150000/" 

## Parameters for data
end_date = dt.today() # use "atexpiration" for expiration date - doesn't seem to work right now
start_date = end_date - pd.Timedelta(days=5*366) # use "firsttraded" for earliest - doesn't seem to work right now
optParams = {
    "from": start_date, 
    "to": end_date,
    "dateformat": "timestamp",
}

## Get Options data from API
r_option = requests.get(url_marketData + optSymbol, optParams)
data_opt = r_option.json()
print("Options data\n", data_opt.keys(), '\n')

Options data
 dict_keys(['s', 'updated', 'bid', 'bidSize', 'mid', 'ask', 'askSize', 'last', 'openInterest', 'volume', 'inTheMoney', 'intrinsicValue', 'extrinsicValue', 'underlyingPrice']) 



In [118]:
## Data to DataFrame
to_keep = ["updated", "bid", "ask", "underlyingPrice"]
to_drop = ["s", "bidSize", "mid", "askSize", "last", "openInterest", "volume", "inTheMoney", "intrinsicValue", "extrinsicValue"]
df_opt = pd.DataFrame.from_dict(data_opt).drop(columns=to_drop).set_index("updated")
df_opt.index = pd.to_datetime(df_opt.index)
display(df_opt)

## Method to view missing dates (non-trading days)
# print(pd.date_range(start = df_opt.index[0], end = df_opt.index[-1]).difference(df_opt.index))

,bid,ask,underlyingPrice
updated,,,
2022-09-12 16:00:00-04:00,43.00,44.30,163.43
2022-09-13 16:00:00-04:00,37.00,39.30,153.84
2022-09-14 16:00:00-04:00,36.75,39.80,155.31
2022-09-15 16:00:00-04:00,36.65,37.30,152.37
2022-09-16 16:00:00-04:00,35.35,37.75,150.70
...,...,...,...
2023-04-25 16:00:00-04:00,36.70,38.00,163.77
2023-04-26 16:00:00-04:00,36.50,37.75,163.76
2023-04-27 16:00:00-04:00,40.40,40.85,168.41


# Calculate volatilities: implied, historical, beta, VIX

## Historical volatility (HV)
$$HV = \frac{\text{underlying standard deviation}}{\text{number of trading days}}$$

In [119]:
## Calculate stdev of underlying
underlying_stdev = df_opt['underlyingPrice'].std()

## Calculate time period of data
## check correct time (i.e. is it counted in full days)
time_period = len(df_opt) # Number of trading days

HV = underlying_stdev / time_period

print(f"Historical volatility = {HV * 100}%")

Historical volatility = 6.438402721226025%


# Calculate risk-free rate of return

In [120]:
## Inflation rate data

## API URL
url_inflation = "https://api.beta.ons.gov.uk/v1/datasets/cpih01"
r_version_data = requests.get(url_inflation + "/editions")

## URL giving links to download datasets in other file formats
url_inflationRate = r_version_data.json()["items"][0]["links"]["latest_version"]["href"]

## Parameter: Aggregate
r_aggregate_options = requests.get(url_inflationRate + "/dimensions/aggregate/options")
# print(json.dumps(r_aggregate_options.json(), indent=2))
aggregate_options = {}
for item in r_aggregate_options.json()["items"]: # iterate through list of dicts
    aggregate_options[item["label"]] = item["option"]
# print(json.dumps(aggregate_options, indent=2))
aggregate_param = aggregate_options["Overall Index"]

## Parameter: Location
r_geography_options = requests.get(url_inflationRate + "/dimensions/geography/options")
# print(json.dumps(r_geography_options.json(), indent=2))
geography_options = {}
for item in r_geography_options.json()["items"]:
    geography_options[item["label"]] = item["option"]
# print(json.dumps(geography_options, indent=2))
geography_param = geography_options["United Kingdom"]

## Parameter: Time
r_time_options = requests.get(url_inflationRate + "/dimensions/time/options")
# print(json.dumps(r_time_options.json(), indent=2))
time_options = []
for item in r_time_options.json()["items"]:
    time_options.append(item["option"])
time_options = sorted(time_options, key=lambda t: dt.strptime(t, "%b-%y"))
# print(time_options)
time_param = time_options[-1]

## Parameters for data
inflParams = { # can use wildcard "*" to retrieve all values for one param at a time
    "time": "*",
    "geography": geography_param,
    "aggregate": aggregate_param,
}

## Get inflation data from API
r_inflation = requests.get(url_inflationRate + "/observations", inflParams)
data_infl = r_inflation.json()
print("Inflation data")
print(data_infl.keys())
# print(json.dumps(data_infl, indent=2))

Inflation data
dict_keys(['@context', 'dimensions', 'limit', 'links', 'observations', 'offset', 'total_observations', 'unit_of_measure'])


In [121]:
## Data to DataFrame

# print(json.dumps(data_infl["observations"], indent=2))
base_measure_year = int(re.search(r"(?<=:\s)(.*?)(?==)", data_infl["unit_of_measure"]).group(0))
base_index = float(re.search(r'(?<==)\w+', obs_measure).group(0))
data_obs = []

for datum in data_infl["observations"]:
    data_obs.append([datum["dimensions"]["Time"]["id"], float(datum["observation"])])

print(f"CPIH index values (Unit of measure given by Index: {base_measure_year} = {base_index})")

column_names = ["Date", "CPIH"]
df_infl = pd.DataFrame(data_obs, columns=column_names).set_index("Date")
df_infl.sort_index(inplace=True, key=lambda x : pd.to_datetime(x, format="%b-%y"))
display(df_infl)

if len(df_infl) != data_infl["total_observations"]:
    print("Mismatch in number of observations transferred to DataFrame.")
    

## Inflation = (Value in Target year - Value in base year) / (Value in Base year) * 100 (%)
## Consider different methods for calculating inflation rate:
## - Base year = Unit of measure year <- use this for now
## - Base year = Year option was first traded
## - Base year = Current year - expected duration of investment

## Reference for comparison:
## https://www.ons.gov.uk/economy/inflationandpriceindices/bulletins/consumerpriceinflation/march2023#:~:text=The%20CPIH%20inflation%20rate%20is,down%20from%2013.4%25%20in%20February.

## Calculate 12-month inflation rate as a standard measure
inflationRate_12M = (df_infl.iloc[-1]["CPIH"] - df_infl.iloc[-13]["CPIH"]) / df_infl.iloc[-13]["CPIH"]
print("12 Month inflation rate: ", f"{inflationRate_12M * 100}%")

CPIH index values (Unit of measure given by Index: 2015 = 100.0)


,CPIH
Date,
Jan-88,46.9
Feb-88,47.0
Mar-88,47.2
Apr-88,47.8
May-88,48.0
...,...
Nov-22,124.8
Dec-22,125.3
Jan-23,124.8


12 Month inflation rate:  8.841201716738196%


In [122]:
## Bond yield data

## Use information from UK Debt Management Office
## https://www.dmo.gov.uk/data/pdfdatareport?reportCode=D1A

## Pick Treasury Gilt 2025 with highest yield as no reason to take a lower yield

giltYield = 0.05

# Risk-free rate

Formulae sources:
- Investopedia
- WallStreetPrep
$$ \text{real risk free rate} = \text{inflation rate} - \text{bond yield (match investment duration)} $$


$$ \text{nominal risk free rate} = \text{(1 + real risk free rate)} \times \text{(1 + inflation rate)} - 1$$

In [123]:
# Risk-Free rate 

realRfRate = inflationRate_12M - giltYield
nomRfRate = (1 + realRfRate) * (1 + inflationRate_12M) - 1

print(f"Real risk-free rate: {realRfRate * 100}%")
print(f"Nominal risk-free rate: {nomRfRate * 100}%")

Real risk-free rate: 3.841201716738195%
Nominal risk-free rate: 13.02201182560001%
